# **Project 1** 

Pick 5 models from RobustBench. Evaluate how accuracy and robustness across classes vary in relationship to the size of the model (i.e., number of parameters).


###Install Robustbench

In [1]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q git+https://github.com/RobustBench/robustbench.git@2d630bc9e8d1cf50d46a4dda65fd36850e3ef769

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 10.8 MB/s eta 0:00:00


###Load Dataset and Model

In [2]:
from robustbench.data import load_cifar10
x_test, y_test = load_cifar10(n_examples=100)

from robustbench.utils import load_model
model = load_model(model_name='Sehwag2021Proxy_R18', threat_model='Linf')


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Download started: path=models/cifar10/Linf/Sehwag2021Proxy_R18.pt (gdrive_id=1-ZgoSlD_AMhtXdnUElilxVXnzK2DcHuu)
Download finished: path=models/cifar10/Linf/Sehwag2021Proxy_R18.pt (gdrive_id=1-ZgoSlD_AMhtXdnUElilxVXnzK2DcHuu)


In [3]:
model.to('cuda:0')
x_test.to('cuda:0')
y_test.to('cuda:0')

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7], device='cuda:0')

In [4]:
for i in range (10):

  class_indices = (y_test == i)
  x_class = x_test[class_indices].to('cuda:0')
  y_class = y_test[class_indices].to('cuda:0')
  print ("data for class /n ",i)
  print(len(y_class))

data for class /n  0
10
data for class /n  1
6
data for class /n  2
8
data for class /n  3
10
data for class /n  4
7
data for class /n  5
8
data for class /n  6
16
data for class /n  7
11
data for class /n  8
13
data for class /n  9
11


In [5]:
import torch
torch.cuda.empty_cache()

###Install AutoAttack

In [6]:
!pip install -q git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack

  Preparing metadata (setup.py) ... done


## Accuracy Evaluation Total Model

In [7]:
adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])

#adversary = AutoAttack(model, norm='Linf', eps=8/255, version='standard')

#adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=['fab-t', 'square'])

adversary.apgd.n_restarts = 1
x_adv = adversary.run_standard_evaluation(x_test, y_test)

using custom version including apgd-ce, apgd-dlr.



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.8/dist-packages/autoattack/checks.py", line 100, in check_dynamic
    sys.settrace(tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.8/dist-packages/autoattack/checks.py", line 102, in check_dynamic
    sys.settrace(None)



initial accuracy: 87.00%
apgd-ce - 1/1 - 29 out of 87 successfully perturbed
robust accuracy after APGD-CE: 58.00% (total time 5.5 s)
apgd-dlr - 1/1 - 4 out of 58 successfully perturbed
robust accuracy after APGD-DLR: 54.00% (total time 9.1 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 54.00%


##Accuracy Evaluation per class

In [8]:
#import torch

num_classes = 10

print("General Robustness \n ")

new_adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
#new_adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=['fab-t', 'square'])
new_adversary.apgd.n_restarts = 1
for i in range (num_classes):

  class_indices = (y_test == i)
  x_class = x_test[class_indices].to('cuda:0')
  y_class = y_test[class_indices].to('cuda:0')
  print ("Robustness for class  ",i)
  x_adv = new_adversary.run_standard_evaluation(x_class, y_class)
  print("##################### /n")

General Robustness 
 
Robustness for class   0
using custom version including apgd-ce, apgd-dlr.
initial accuracy: 90.00%
apgd-ce - 1/1 - 2 out of 9 successfully perturbed
robust accuracy after APGD-CE: 70.00% (total time 1.1 s)
apgd-dlr - 1/1 - 0 out of 7 successfully perturbed
robust accuracy after APGD-DLR: 70.00% (total time 2.1 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 70.00%
##################### /n
Robustness for class   1
using custom version including apgd-ce, apgd-dlr.
initial accuracy: 83.33%
apgd-ce - 1/1 - 4 out of 5 successfully perturbed
robust accuracy after APGD-CE: 16.67% (total time 0.9 s)
apgd-dlr - 1/1 - 0 out of 1 successfully perturbed
robust accuracy after APGD-DLR: 16.67% (total time 2.7 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 16.67%
##################### /n
Robustness for class   2
using custom version including apgd-ce, apgd-dlr.
initial accuracy: 7